In [65]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb

In [22]:
data_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [23]:
data_df.columns

Index(['ID', 'flow_duration', 'Header_Length', 'Protocol type', 'Duration',
       'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
       'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
       'ece_flag_number', 'cwr_flag_number', 'ack_count', 'syn_count',
       'fin_count', 'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet',
       'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC',
       'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number',
       'Magnitue', 'Radius', 'Covariance', 'Variance', 'Weight', 'Label'],
      dtype='object')

In [24]:
test_df.columns

Index(['ID', 'flow_duration', 'Header_Length', 'Protocol type', 'Duration',
       'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
       'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
       'ece_flag_number', 'cwr_flag_number', 'ack_count', 'syn_count',
       'fin_count', 'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet',
       'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC',
       'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number',
       'Magnitue', 'Radius', 'Covariance', 'Variance', 'Weight'],
      dtype='object')

In [25]:
data_df.fillna(0,inplace=True)

In [26]:
test_df.fillna(0,inplace=True)

In [27]:
data_df

,ID,flow_duration,Header_Length,Protocol type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,Label
0,769866,0.000000,54.00,6.00,64.00,2.890680,2.890680,0.0,0.0,0.0,...,0.000000,54.00,8.294698e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DoS-TCP_Flood
1,1859874,0.000892,54.58,0.00,64.00,28.594443,28.594443,0.0,0.0,1.0,...,0.000000,54.00,8.297298e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DoS-SYN_Flood
2,396092,0.000000,0.00,1.00,64.00,23.716061,23.716061,0.0,0.0,0.0,...,0.000000,0.00,8.312478e+07,9.5,9.165151,0.000000,0.000000,0.00,141.55,DDoS-ICMP_Flood
3,179708,109.146727,1992.80,8.20,50.10,41.102103,41.102103,0.0,0.0,0.0,...,305.527954,223.00,0.000000e+00,13.5,19.438032,433.145956,98347.063168,1.00,244.60,Recon-PortScan
4,1527289,0.025221,58.53,5.95,65.91,25.300629,25.300629,0.0,1.0,0.0,...,1.158718,56.76,8.334493e+07,9.5,10.420536,1.657137,61.278209,0.04,141.55,DDoS-RSTFINFlood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1950133,913815,2.126253,0.00,16.66,64.00,1239.345221,1239.345221,0.0,0.0,0.0,...,46.211504,518.73,8.378464e+07,9.5,32.752232,0.000000,6468.374735,0.35,141.55,Mirai-udpplain
1950134,2150643,1.340738,77.76,6.00,64.00,0.656392,0.656392,0.0,0.0,1.0,...,0.000000,54.00,8.336185e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-SynonymousIP_Flood
1950135,368961,189.003258,12885.50,0.00,100.60,21.719009,21.719009,0.0,0.0,0.0,...,824.460757,133.00,1.668479e+08,13.5,30.729074,1168.193676,686325.224261,1.00,244.60,Recon-HostDiscovery
1950136,250960,298.879657,7006.50,7.60,147.50,0.451747,0.451747,0.0,0.0,0.0,...,52.308068,105.30,4.257479e-02,5.5,14.227318,73.974779,3404.204759,0.90,38.50,Recon-PortScan


In [28]:
test_df

,ID,flow_duration,Header_Length,Protocol type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,...,AVG,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight
0,557973,5.512903,7551111.00,6.00,50.00,1071.939766,1071.939766,0.0,0.0,0.0,...,1320.323802,489.873763,1514.00,1.676389e+08,13.5,0.000000,694.197492,2.424091e+05,1.00,244.60
1,341704,0.000000,741.86,5.88,62.72,0.485594,0.485594,0.0,0.0,0.0,...,918.596271,545.057732,926.32,8.333613e+07,9.5,42.701969,770.726968,3.129036e+05,0.95,141.55
2,2050100,223.220984,8405.20,9.30,175.20,3.722975,3.722975,0.0,0.0,0.0,...,131.960660,92.642629,0.00,1.676388e+08,13.5,16.267696,0.000000,8.612926e+03,1.00,0.00
3,1515342,41.144953,2183780.60,17.00,51.30,95.883044,95.883044,0.0,0.0,0.0,...,465.310675,163.896434,459.70,1.722813e-04,5.5,0.000000,231.784560,4.170580e+04,0.90,38.50
4,2565329,0.000000,1150033.90,6.00,63.60,69.455512,69.455512,0.0,0.0,0.0,...,2807.201905,1197.260005,0.00,1.290417e-03,5.5,74.596326,1693.181337,1.600649e+06,0.90,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835769,525880,0.024796,23025.00,17.00,64.00,18561.111777,18561.111777,0.0,0.0,0.0,...,50.000000,0.000000,50.00,8.302865e+07,9.5,10.000000,0.000000,0.000000e+00,0.00,141.55
835770,1906010,17.023402,591162.54,6.73,67.62,40.023840,40.023840,0.0,0.0,0.0,...,613.261716,453.792424,677.02,8.334043e+07,9.5,32.198541,641.148276,2.687407e+05,0.94,141.55
835771,338122,0.049538,745.84,5.88,66.54,1.796532,1.796532,0.0,0.0,0.0,...,906.950758,0.000000,890.80,8.333617e+07,9.5,42.430498,773.522975,3.152259e+05,0.95,141.55
835772,2093607,3.044790,1188495.65,17.00,64.64,688.658836,688.658836,0.0,0.0,0.0,...,553.166679,3.477494,550.16,8.375905e+07,9.5,33.261049,4.946983,3.060530e+02,0.04,141.55


In [29]:
data_df['Label'].value_counts()

Label
MITM-ArpSpoofing           84260
DDoS-TCP_Flood             84208
Recon-HostDiscovery        84139
DoS-UDP_Flood              84139
DDoS-ICMP_Fragmentation    84119
DoS-TCP_Flood              84086
BenignTraffic              84085
DDoS-SYN_Flood             84051
Mirai-greip_flood          84050
DDoS-PSHACK_Flood          84047
DDoS-SynonymousIP_Flood    84021
Mirai-greeth_flood         84018
DDoS-ACK_Fragmentation     84015
DoS-SYN_Flood              83971
DDoS-RSTFINFlood           83949
DDoS-UDP_Flood             83905
DDoS-UDP_Fragmentation     83895
DNS_Spoofing               83746
Mirai-udpplain             83653
DDoS-ICMP_Flood            83587
Recon-OSScan               69603
Recon-PortScan             58311
DoS-HTTP_Flood             50629
VulnerabilityScan          26378
DDoS-HTTP_Flood            20338
DDoS-SlowLoris             16573
DictionaryBruteForce        9213
BrowserHijacking            4134
CommandInjection            3821
SqlInjection                3748
XSS 

In [30]:
X_columns = [
    'ID', 'flow_duration', 'Header_Length', 'Protocol type', 'Duration',
       'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
       'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
       'ece_flag_number', 'cwr_flag_number', 'ack_count', 'syn_count',
       'fin_count', 'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet',
       'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC',
       'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number',
       'Magnitue', 'Radius', 'Covariance', 'Variance', 'Weight'
]

# DF column used for the attack labels
y_column = 'Label'

In [31]:
# Creating a dictionary of attack types for 33 attack classes + 1 for benign traffic
dict_34_classes = {'BenignTraffic': 0 ,                                                                                                                         # Benign
                    'DDoS-RSTFINFlood' :1, 'DDoS-PSHACK_Flood':2,  'DDoS-SYN_Flood':3, 'DDoS-UDP_Flood':4, 'DDoS-TCP_Flood':5, 
                    'DDoS-ICMP_Flood':6, 'DDoS-SynonymousIP_Flood':7, 'DDoS-ACK_Fragmentation':8, 'DDoS-UDP_Fragmentation':9, 'DDoS-ICMP_Fragmentation':10, 
                    'DDoS-SlowLoris':11, 'DDoS-HTTP_Flood':12, 'DoS-UDP_Flood':13, 'DoS-SYN_Flood':14, 'DoS-TCP_Flood':15, 'DoS-HTTP_Flood':16,                 # DDoS and DoS
                    'Mirai-greeth_flood': 17, 'Mirai-greip_flood': 18, 'Mirai-udpplain': 19,                                                                    # Mirai 
                    'Recon-PingSweep': 20, 'Recon-OSScan': 21, 'Recon-PortScan': 22, 'VulnerabilityScan': 23, 'Recon-HostDiscovery': 24,                        # Reconnaissance
                    'DNS_Spoofing': 25, 'MITM-ArpSpoofing': 26,                                                                                                 # Spoofing
                    'BrowserHijacking': 27, 'Backdoor_Malware': 28, 'XSS': 29, 'Uploading_Attack': 30, 'SqlInjection': 31, 'CommandInjection': 32,              # Web
                    'DictionaryBruteForce': 33}  

# Binary classes
dict_2_classes = {  0: 0 ,                                                                                                                                      # Benign
                    1 :1, 2:1,  3:1, 4:1, 5:1, 6:1, 7:1, 8:1, 9:1, 10:1, 11:1, 12:1, 13:1, 14:1, 15:1, 16:1,                                                    # DDoS and DoS  
                    17: 1, 18: 1, 19: 1,                                                                                                                        # Mirai 
                    20: 1, 21: 1, 22: 1, 23: 1, 24: 1,                                                                                                          # Reconnaissance
                    25: 1, 26: 1,                                                                                                                               # Spoofing
                    27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1,                                                                                                   # Web
                    33: 1}    

In [32]:
print(data_df.shape)
# Take a random sample of 5% of the rows
# data_df = data_df.sample(random_state=42)
data_df['Label'] = data_df['Label'].map(dict_34_classes)
data_df['Label'] = data_df['Label'].map(dict_2_classes)
num_unique_classes = len(data_df[y_column].unique())
print("unique classess: ", num_unique_classes)

(1950138, 48)
unique classess:  2


In [33]:
scaler = StandardScaler()
data_df[X_columns] = scaler.fit_transform(data_df[X_columns])

In [34]:
print("Data 1 size: {}".format(data_df.shape))
print(data_df.info())

Data 1 size: (1950138, 48)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1950138 entries, 0 to 1950137
Data columns (total 48 columns):
 #   Column           Dtype  
---  ------           -----  
 0   ID               float64
 1   flow_duration    float64
 2   Header_Length    float64
 3   Protocol type    float64
 4   Duration         float64
 5   Rate             float64
 6   Srate            float64
 7   Drate            float64
 8   fin_flag_number  float64
 9   syn_flag_number  float64
 10  rst_flag_number  float64
 11  psh_flag_number  float64
 12  ack_flag_number  float64
 13  ece_flag_number  float64
 14  cwr_flag_number  float64
 15  ack_count        float64
 16  syn_count        float64
 17  fin_count        float64
 18  urg_count        float64
 19  rst_count        float64
 20  HTTP             float64
 21  HTTPS            float64
 22  DNS              float64
 23  Telnet           float64
 24  SMTP             float64
 25  SSH              float64
 26  IRC            

In [35]:
data_df['Label'].value_counts()

Label
1    1866053
0      84085
Name: count, dtype: int64

In [36]:
data_df.to_csv('./preprocessed_data.csv')

In [37]:
data_df

,ID,flow_duration,Header_Length,Protocol type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,Label
0,-0.775052,-0.060556,-0.280110,-0.375805,-0.081082,-0.079045,-0.079505,-0.00088,-0.212622,-0.425158,...,-0.506799,-0.595818,0.170033,0.001266,-0.603357,-0.506452,-0.234179,-0.846197,0.001260,1
1,0.580621,-0.060555,-0.280110,-0.897363,-0.081082,-0.078712,-0.079168,-0.00088,-0.212622,2.352064,...,-0.506799,-0.595818,0.170587,0.001266,-0.603357,-0.506452,-0.234179,-0.846197,0.001260,1
2,-1.239926,-0.060556,-0.280176,-0.810436,-0.081082,-0.078775,-0.079232,-0.00088,-0.212622,-0.425158,...,-0.506799,-0.720115,0.173818,0.001266,-0.684193,-0.506452,-0.234179,-0.846197,0.001260,1
3,-1.509049,0.049695,-0.277754,-0.184568,-0.476012,-0.078550,-0.079004,-0.00088,-0.212622,-0.425158,...,0.510026,-0.206812,-1.595829,1.988084,-0.007486,0.513575,-0.055861,1.472861,1.989305,1
4,0.166976,-0.060531,-0.280105,-0.380152,-0.026814,-0.078755,-0.079211,-0.00088,4.703177,-0.425158,...,-0.502943,-0.589465,0.178505,0.001266,-0.601497,-0.502549,-0.234068,-0.753435,0.001260,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1950133,-0.596019,-0.058408,-0.280176,0.550828,-0.081082,-0.063034,-0.063317,-0.00088,-0.212622,-0.425158,...,-0.353003,0.473901,0.187866,0.001266,0.869563,-0.506452,-0.222451,-0.034527,0.001260,1
1950134,0.942259,-0.059202,-0.280081,-0.375805,-0.081082,-0.079074,-0.079534,-0.00088,-0.212622,2.352064,...,-0.506799,-0.595818,0.178865,0.001266,-0.603357,-0.506452,-0.234179,-0.846197,0.001260,1
1950135,-1.273669,0.130360,-0.264517,-0.897363,0.958806,-0.078801,-0.079258,-0.00088,-0.212622,-0.425158,...,2.237083,-0.413975,1.956204,1.988084,0.736291,2.244558,1.010230,1.472861,1.989305,1
1950136,-1.420431,0.241349,-0.271662,-0.236723,2.291341,-0.079076,-0.079536,-0.00088,-0.212622,-0.425158,...,-0.332713,-0.477735,-1.595829,-1.985552,-0.350732,-0.332247,-0.228007,1.240955,-1.986785,1


In [38]:
data_df["Label"].value_counts()

Label
1    1866053
0      84085
Name: count, dtype: int64

TRAIN MODEL

# Create model Xgboost


In [39]:
X = data_df[X_columns]
y = data_df[y_column]

# Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

X_train:  (1560110, 47)
X_test:  (390028, 47)
y_train:  (1560110,)
y_test:  (390028,)


In [67]:
# Train the model
model = xgb.XGBClassifier()
model.fit(X_train, y_train)
print("Done training")
# Make predictions on the test set
y_pred = model.predict(X_test)
print("Done predicting")

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
f1score  = f1_score(y_test, y_pred, average='weighted')
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("f1score: %.2f%%" % (f1score * 100.0))
print("Done")

Done training
Done predicting
Accuracy: 98.58%
f1score: 98.55%
Done


# Turnning Hyperparameter

In [44]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [73]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

[CV 1/3] END colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=1.0;, score=0.994 total time=   3.3s
[CV 3/3] END colsample_bytree=0.8, gamma=1.5, max_depth=5, min_child_weight=1, subsample=0.6;, score=0.995 total time=   4.0s
[CV 1/3] END colsample_bytree=0.6, gamma=5, max_depth=5, min_child_weight=5, subsample=1.0;, score=0.995 total time=   3.5s
[CV 1/3] END colsample_bytree=1.0, gamma=1, max_depth=4, min_child_weight=1, subsample=0.8;, score=0.995 total time=   3.3s
[CV 3/3] END colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=1.0;, score=0.994 total time=   3.4s
[CV 1/3] END colsample_bytree=0.8, gamma=1, max_depth=5, min_child_weight=5, subsample=0.8;, score=0.995 total time=   3.9s
[CV 2/3] END colsample_bytree=0.6, gamma=5, max_depth=5, min_child_weight=5, subsample=1.0;, score=0.995 total time=   3.5s
[CV 2/3] END colsample_bytree=1.0, gamma=1, max_depth=4, min_child_weight=1, subsample=0.8;, score=0.995 total time=   3.4s
[CV 2/

In [74]:
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=param_comb, scoring='f1', n_jobs=4, cv=skf.split(X_train,y_train), verbose=3, random_state=1001 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 3 folds for each of 5 candidates, totalling 15 fits

 Time taken: 0 hours 0 minutes and 17.54 seconds.


In [75]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)


 All results:
{'mean_fit_time': array([3.10371772, 3.81268462, 3.77789712, 3.3733983 , 3.2815601 ]), 'std_fit_time': array([0.02311105, 0.05250756, 0.07144206, 0.07527206, 0.03504773]), 'mean_score_time': array([0.17568429, 0.19251243, 0.18487302, 0.17485833, 0.15591852]), 'std_score_time': array([0.00493704, 0.00119235, 0.01277162, 0.02161493, 0.01903397]), 'param_subsample': masked_array(data=[1.0, 0.6, 0.8, 1.0, 0.8],
             mask=[False, False, False, False, False],
       fill_value=1e+20), 'param_min_child_weight': masked_array(data=[5, 1, 5, 5, 1],
             mask=[False, False, False, False, False],
       fill_value=999999), 'param_max_depth': masked_array(data=[3, 5, 5, 5, 4],
             mask=[False, False, False, False, False],
       fill_value=999999), 'param_gamma': masked_array(data=[5.0, 1.5, 1.0, 5.0, 1.0],
             mask=[False, False, False, False, False],
       fill_value=1e+20), 'param_colsample_bytree': masked_array(data=[1.0, 0.8, 0.8, 0.6, 1.0],
  

In [83]:
# Tạo DMatrix từ dữ liệu 
dtrain = xgb.DMatrix(X_train, label=y_train) 
# Đặt tham số 
param = { 'objective': 'binary:logistic', 'subsample': 0.8, 'min_child_weight': 5, 'max_depth': 5, 'gamma': 1, 'colsample_bytree': 0.8 } 
# Huấn luyện mô hình 
fmodel = xgb.train(param, dtrain, num_boost_round=100)

In [91]:
# Tạo DMatrix từ tập kiểm tra 
dtest = xgb.DMatrix(X_test, label=y_test) 
# Dự đoán kết quả 
fy_pred = fmodel.predict(dtest) 
fy_pred = [1 if y > 0.5 else 0 for y in fy_pred] 
# Ngưỡng 0.5 để phân loại # Tính Accuracy 
faccuracy = accuracy_score(y_test, fy_pred) 
print("Accuracy:", faccuracy * 100 , "%") 
# Tính F1-score 
ff1_score = f1_score(y_test, fy_pred) 
print("F1 Score:", ff1_score * 100 , "%") 

Accuracy: 98.53497697601198 %
F1 Score: 99.23637189784436 %
